In [ ]:
from pyomo.environ import *

model = ConcreteModel()

model.VEGETABLES = Set(initialize=['H1', 'H2', 'H3'])
model.X = RangeSet(0, 5)
model.TIME = RangeSet(0, 10)

model.growth = Param(model.VEGETABLES, initialize={'H1': 2, 'H2': 4, 'H3': 5})

model.scheduling = Var(model.X, model.VEGETABLES, model.TIME, domain=Binary)

def objective_rule(model):
    return sum(model.scheduling[x, h, t] / model.growth[h]
               for x in model.X for h in model.VEGETABLES for t in model.TIME)

model.objective = Objective(rule=objective_rule, sense=maximize)

def supply_unique_vegetables(model, h, t):
    return sum(model.scheduling[x, h, t] for x in model.X) <= 1

model.demand_constraint = Constraint(model.VEGETABLES, model.TIME, rule=supply_unique_vegetables)

solver = SolverFactory('glpk')
solver.options['tmlim'] = 60
results = solver.solve(model)

if results.solver.termination_condition == TerminationCondition.optimal:
    print("Optimal solution found")
    for x in model.X:
        for h in model.VEGETABLES:
            for t in model.TIME:
                print(f"Quantity from {x} to {h} at time {t}: {model.scheduling[x, h, t].value}")
    print("Total objective value:", model.objective())
else:
    print("Solver did not converge to an optimal solution")



Optimal solution found
Quantity from 0 to H1 at time 0: 1.0
Quantity from 0 to H1 at time 1: 1.0
Quantity from 0 to H1 at time 2: 1.0
Quantity from 0 to H1 at time 3: 1.0
Quantity from 0 to H1 at time 4: 1.0
Quantity from 0 to H1 at time 5: 1.0
Quantity from 0 to H1 at time 6: 1.0
Quantity from 0 to H1 at time 7: 1.0
Quantity from 0 to H1 at time 8: 1.0
Quantity from 0 to H1 at time 9: 1.0
Quantity from 0 to H1 at time 10: 1.0
Quantity from 0 to H2 at time 0: 1.0
Quantity from 0 to H2 at time 1: 1.0
Quantity from 0 to H2 at time 2: 1.0
Quantity from 0 to H2 at time 3: 1.0
Quantity from 0 to H2 at time 4: 1.0
Quantity from 0 to H2 at time 5: 1.0
Quantity from 0 to H2 at time 6: 1.0
Quantity from 0 to H2 at time 7: 1.0
Quantity from 0 to H2 at time 8: 1.0
Quantity from 0 to H2 at time 9: 1.0
Quantity from 0 to H2 at time 10: 1.0
Quantity from 0 to H3 at time 0: 1.0
Quantity from 0 to H3 at time 1: 1.0
Quantity from 0 to H3 at time 2: 1.0
Quantity from 0 to H3 at time 3: 1.0
Quantity from

In [2]:
init_dict = {(x, h, t): 0 for x in range(4) for h in ['H1', 'H2', 'H3'] for t in range(7)}

In [3]:
for t in range(1,4):
    init_dict[0, 'H1', t] = 1
    init_dict[1, 'H2', t] = 1
    init_dict[2, 'H3', t] = 1
    init_dict[3, 'H2', t] = 1
init_dict[0, 'H1', 4] = 1
init_dict[0, 'H1', 5] = 1

In [10]:
from pyomo.environ import *
from pyomo.core.util import prod
from math import ceil
import sys
sys.path.append('/home/marnugue/Documents/crop_scheduling_optimization')
from src.utils.epsilon_constraint import EpsilonConstraintAugment
from src.utils.max_constraint import max_constraints
# Create an instance of a concrete model
model = ConcreteModel()

# Sets
model.VEGETABLES = Set(initialize=['H1', 'H2', 'H3'])
model.X = RangeSet(0,3)
model.TIME = RangeSet(0,6)
# Parameters
# Transportation costs from factories to warehouses
model.growth = Param(model.VEGETABLES, initialize={
    'H1': 5,
    'H2': 3,
    'H3': 3
})

# Variables
model.scheduling = Var(model.X, model.VEGETABLES, model.TIME, initialize=init_dict, domain=Binary)

# max_var_names = []
for x in model.X:
    for h in model.VEGETABLES:
        for t in model.TIME:
            if t>0:
                var1_name = f'scheduling_{x}_{h}_{t}'
                var2_name = f'scheduling_{x}_{h}_{t-1}'
                max_constraints(model, model.scheduling[x,h,t], model.scheduling[x,h,t - 1],  var1_str_name=var1_name,
                    var2_str_name=var2_name)

def diff_expr(model, x, h, t):
    if t> 0:
        var_name_1 = f'scheduling_{x}_{h}_{t}'
        var_name_2 = f'scheduling_{x}_{h}_{t-1}'
        return (1 - (model.scheduling[x,h,t -1]*model.scheduling[x,h,t]))*(getattr(model, f'max_value_{var_name_1}_{var_name_2}'))*model.scheduling[x,h,t]
    else:
        return 0
    
model.exp1 = Expression(model.X, model.VEGETABLES, model.TIME, rule=diff_expr)
def concat_expr(model, x,h,t):
    
    if t > model.growth[h] + 1:
        return prod(model.scheduling[x,h,t - t_i] for t_i in range(model.growth[h] + 1))
    else:
        return 0
    
model.exp2 = Expression(model.X, model.VEGETABLES, model.TIME, rule=concat_expr)
    
def _exp_plant_vegetable_bin(model, x, h, t):
    return  model.exp1[x, h, t]

model.plant_vegetable_bin = Expression(model.X, model.VEGETABLES, model.TIME, rule=_exp_plant_vegetable_bin )

# def _exp_plant_vegetable_bin(model, x, h, t):
    
#     if t == 0:
#         return Expression.Skip
    
#     if (value(model.scheduling[x,h,t-1]) != value(model.scheduling[x,h,t])) or\
#         (t > model.growth[h] and all([value(model.scheduling[x,h,t - t_i]) for t_i in range(model.growth[h])])):
#         return 1
    
#     return 0

# model.plant_vegetable_bin =  Var(model.X, model.VEGETABLES, model.TIME, initialize=0) 

# Constraints
def supply_time_growth(model, x, h, t):
    
    if t == 0:
        return model.scheduling[x, h,  t] == 0
    
    post_hours = min(t + model.growth[h] - 1, max(model.TIME))
    
    post_sum = sum(model.scheduling[x, h, t_i] for t_i in range(t, post_hours + 1))
    
    if t + model.growth[h] - 1 > post_hours:
        max_value = (max(model.TIME) - t)  + 1
    else:
        max_value = model.growth[h] + 1
    
    # right_side = value(min(model.min_variable[x, h, t]))
    
    return  ceil(model.plant_vegetable_bin[x,h,t]) * (max_value - post_sum)  <= 1

def vegetable_production(model, h):
    return sum(model.scheduling[x, h, t] for x in model.X for t in model.TIME)/model.growth[h]

def supply_unique_vegetables(model, x, t):
    return sum(model.scheduling[x, h, t] for h in model.VEGETABLES) <= 1  # Constraint on warehouses' demand

# interchange vegetables

# def interchange_vegetables(model, x, h, t):
#     post_hours = min(t + model.growth[h], max(model.TIME))
    
#     return model.plant_vegetable_bin[x, h, t] * model.scheduling[x, h, post_hours] == 0

# model.interchange_constraint = Constraint(model.X, model.VEGETABLES, model.TIME, rule=interchange_vegetables)

model.supply_constraint = Constraint(model.X, model.VEGETABLES, model.TIME, rule=supply_time_growth)
model.demand_constraint = Constraint(model.X, model.TIME, rule=supply_unique_vegetables)

model.abs_diff = Var(range(len(model.VEGETABLES)**2), initialize=0, domain=NonNegativeReals)

model.expr_vegetables = Expression(model.VEGETABLES, initialize=vegetable_production)

# Define constraints to linearize absolute differences
def abs_diff_constraint_rule(model, i, h1, h2):
    return model.abs_diff[i] >= model.expr_vegetables[h1] - model.expr_vegetables[h2]

model.abs_diff_constraint_pos = Constraint(range(len(model.VEGETABLES)**2),
                                           model.VEGETABLES,
                                           model.VEGETABLES,
                                           rule=abs_diff_constraint_rule)

def abs_diff_constraint_rule_neg(m, i, h1, h2):
    return m.abs_diff[i] >= - model.expr_vegetables[h1] +  model.expr_vegetables[h2]

model.abs_diff_constraint_neg = Constraint(range(len(model.VEGETABLES)**2),
                                           model.VEGETABLES,
                                           model.VEGETABLES,
                                           rule=abs_diff_constraint_rule_neg)


# Objective function
def production(model):
    return sum(model.scheduling[x, h, t]/ model.growth[h]
               for x in model.X for h in model.VEGETABLES for t in model.TIME)

# def balanced_production(model):
#     return - sum(abs(model.exp_vegetables[v1] - model.exp_vegetables[v2]) for v1 in model.VEGETABLES for v2 in model.VEGETABLES)

def linear_balanced_production(model):
    return - sum(model.abs_diff[i] for i in range(len(model.VEGETABLES)**2))

model.production = Objective(rule=production, sense=maximize)
model.demmand = Objective(rule=linear_balanced_production, sense=maximize)

model.demmand.deactivate()

# eps = EpsilonConstraintAugment(model, solver='cbc')
# solutions = eps.execute(time_limit=10)

# model.objective = Multi

# Solve the model
solver = SolverFactory('ipopt')  # Using GLPK solver
results = solver.solve(model)
# solver = SolverFactory('mindtpy')
#         # solver.options['seconds'] = timelimit
# status = solver.solve(model, mip_solver='cbc',
#                               nlp_solver='ipopt')
# # Display results
# if results.solver.termination_condition == TerminationCondition.optimal:
#     print("Optimal solution found")
#     for x in model.X:
#         for h in model.VEGETABLES:
#             for t in model.TIME:
#                 print(f"Quantity transported from {x} to {h} {t}: {model.scheduling[x, h, t].value}")
    # for h in model.VEGETABLES:b
    
    #     print(model.param_vegetables[h])
    # print("Total cost:", model.demmand())
# else:
#     print("Solver did not converge to an optimal solution")


ERROR: Rule failed when generating expression for Constraint supply_constraint
with index (0, 'H1', 1): TypeError: Implicit conversion of Pyomo numeric value
(plant_vegetable_bin[0,H1,1]) to float is disabled. This error is often the
result of using Pyomo components as arguments to one of the Python built-in
math module functions when defining expressions. Avoid this error by using
Pyomo-provided math functions or explicitly resolving the numeric value using
the Pyomo value() function.
ERROR: Constructing component 'supply_constraint' from data=None failed:
TypeError: Implicit conversion of Pyomo numeric value
(plant_vegetable_bin[0,H1,1]) to float is disabled. This error is often the
result of using Pyomo components as arguments to one of the Python built-in
math module functions when defining expressions. Avoid this error by using
Pyomo-provided math functions or explicitly resolving the numeric value using
the Pyomo value() function.


TypeError: Implicit conversion of Pyomo numeric value (plant_vegetable_bin[0,H1,1]) to float is disabled.
This error is often the result of using Pyomo components as arguments to
one of the Python built-in math module functions when defining
expressions. Avoid this error by using Pyomo-provided math functions or
explicitly resolving the numeric value using the Pyomo value() function.

In [8]:
import pandas as pd
df = pd.DataFrame(columns=['x','h','t','value'])
idx=0
for x in model.X:
    for h in model.VEGETABLES:
        for t in model.TIME:
            idx+=1    
            df.loc[idx,:] = [x,h,t,ceil(model.scheduling[x, h, t].value)]

In [9]:
df

,x,h,t,value
1,0,H1,0,0
2,0,H1,1,1
3,0,H1,2,1
4,0,H1,3,1
5,0,H1,4,1
6,0,H1,5,1
7,0,H1,6,1
8,0,H2,0,0
9,0,H2,1,1
10,0,H2,2,1


In [5]:
import plotly.express as px
import pandas as pd
import re
regex = r"scheduling\[(\w+),(\w+),(\w+)\]"

In [7]:
df = pd.DataFrame(columns=['x','h','t','value'])
solution = solutions[0].decision_vars

In [7]:
pd.options.display.max_rows = 90
df

,x,h,t,value
1,0,H1,0,-0.0
2,0,H1,1,0.0
3,0,H1,2,0.0
4,0,H1,3,0.0
5,0,H1,4,0.0
6,0,H1,5,0.0
7,0,H1,6,0.0
8,0,H2,0,-0.0
9,0,H2,1,0.235582
10,0,H2,2,0.000015


In [70]:
for i in range(4):
    print(solutions[i].obj_values)

{'production': 8.0, 'demmand': -36.0}
{'production': 7.466666666666667, 'demmand': -22.799999699999997}
{'production': 7.199999999999999, 'demmand': -16.2}
{'production': 6.8, 'demmand': -7.80000003}


In [8]:
idx = 0 
for  key , value in  solution.items():
    columns = re.findall(regex, key)
    if columns:
        columns = columns[0]
    else:
        continue
    df.loc[idx,:] = list(columns) + [value]
    idx+=1
# df = df[df['x']=='0']

In [9]:
df.groupby('h').sum()

,x,t,value
h,,,
H1,0000000111111122222223333333,0123456012345601234560123456,0.0
H2,0000000111111122222223333333,0123456012345601234560123456,0.0
H3,0000000111111122222223333333,0123456012345601234560123456,0.0


In [11]:
pd.options.display.max_rows = 90

In [26]:
fig = px.bar(df,x='t',y='value', color='x', hover_data=['h'],labels={'h':'h'})
fig.update_layout(bargap=0)
fig.show()

In [76]:
regex = r"plant_vegetable_bin\[(\w+),(\w+),(\w+)\]"
df = pd.DataFrame(columns=['x','h','t','value'])
solution = solutions[4].decision_vars
idx = 0 
for  key , value in  solution.items():
    columns = re.findall(regex, key)
    if columns:
        columns = columns[0]
    else:
        continue
    df.loc[idx,:] = list(columns) + [value]
    idx+=1
fig = px.bar(df,x='t',y='value', color='x', hover_data=['h'],labels={'h':'h'})
fig.update_layout(bargap=0)
fig.show()

In [17]:
plant_vegetable_bin

['0', 'H1', '0']

In [23]:
solutions[0].obj_values

{'production': 21.99999999999999, 'demmand': -0.0}

In [39]:
for x in model.X:
    for h in model.VEGETABLES:
        for t in model.TIME:
            print(f"Quantity transported from {x} to {h} {t}: {model.scheduling[x, h, t].value}")

Quantity transported from 0 to H1 0: 1.0
Quantity transported from 0 to H1 1: 1.0
Quantity transported from 0 to H1 2: 1.0
Quantity transported from 0 to H1 3: 1.0
Quantity transported from 0 to H1 4: 1.0
Quantity transported from 0 to H1 5: 1.0
Quantity transported from 0 to H1 6: 1.0
Quantity transported from 0 to H1 7: 1.0
Quantity transported from 0 to H1 8: 1.0
Quantity transported from 0 to H1 9: 1.0
Quantity transported from 0 to H1 10: 1.0
Quantity transported from 0 to H2 0: 0.0
Quantity transported from 0 to H2 1: 0.0
Quantity transported from 0 to H2 2: 0.0
Quantity transported from 0 to H2 3: 0.0
Quantity transported from 0 to H2 4: 0.0
Quantity transported from 0 to H2 5: 0.0
Quantity transported from 0 to H2 6: 0.0
Quantity transported from 0 to H2 7: 0.0
Quantity transported from 0 to H2 8: 0.0
Quantity transported from 0 to H2 9: 0.0
Quantity transported from 0 to H2 10: 0.0
Quantity transported from 0 to H3 0: 0.0
Quantity transported from 0 to H3 1: 0.0
Quantity trans

In [15]:
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Sets, Parameters, Decision Variables, Constraints defined as before...

# Objective Functions
def cost_rule(model):
    return sum(model.Cost[p] * model.Quantity[p] for p in model.Products)
model.CostObjective = Objective(rule=cost_rule, sense=minimize)

def efficiency_rule(model):
    return sum(model.Efficiency[p] * model.Quantity[p] for p in model.Products)
model.EfficiencyObjective = Objective(rule=efficiency_rule, sense=maximize)

# Solve the multi-objective optimization problem
opt = SolverFactory('cbc')
opt.solve(model, strategy='nondominated')

# Display Results
print("Optimal Quantities:")
for p in model.Products:
    print(f"{p}: {model.Quantity[p].value}")

print("Total Cost:", model.CostObjective())
print("Total Efficiency:", model.EfficiencyObjective())


ERROR: Rule failed when generating expression for Objective CostObjective with
index None: AttributeError: 'ConcreteModel' object has no attribute 'Products'
ERROR: Constructing component 'CostObjective' from data=None failed:
AttributeError: 'ConcreteModel' object has no attribute 'Products'


AttributeError: 'ConcreteModel' object has no attribute 'Products'

# Epsilon implementation/

In [ ]:
from pyomo.environ import *
import matplotlib.pyplot as plt


# max f1 = X1 <br>
# max f2 = 3 X1 + 4 X2 <br>
# st  X1 <= 20 <br>
#     X2 <= 40 <br>
#     5 X1 + 4 X2 <= 200 <br>

model = ConcreteModel()

model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)

model.C1 = Constraint(expr = model.X1 <= 20)
model.C2 = Constraint(expr = model.X2 <= 40)
model.C3 = Constraint(expr = 5 * model.X1 + 4 * model.X2 <= 200)

model.f1 = Var()
model.f2 = Var()
model.C_f1 = Constraint(expr= model.f1 == model.X1)
model.C_f2 = Constraint(expr= model.f2 == 3 * model.X1 + 4 * model.X2)
model.O_f1 = Objective(expr= model.f1  , sense=maximize)
model.O_f2 = Objective(expr= model.f2  , sense=maximize)

model.O_f2.deactivate()

solver = SolverFactory('cplex')
solver.solve(model);

print( '( X1 , X2 ) = ( ' + str(value(model.X1)) + ' , ' + str(value(model.X2)) + ' )')
print( 'f1 = ' + str(value(model.f1)) )
print( 'f2 = ' + str(value(model.f2)) )
f2_min = value(model.f2)


# ## max f2

model.O_f2.activate()
model.O_f1.deactivate()

solver = SolverFactory('cplex')
solver.solve(model);

print( '( X1 , X2 ) = ( ' + str(value(model.X1)) + ' , ' + str(value(model.X2)) + ' )')
print( 'f1 = ' + str(value(model.f1)) )
print( 'f2 = ' + str(value(model.f2)) )
f2_max = value(model.f2)


# ## apply normal $\epsilon$-Constraint

model.O_f1.activate()
model.O_f2.deactivate()

model.e = Param(initialize=0, mutable=True)

model.C_epsilon = Constraint(expr = model.f2 == model.e)

solver.solve(model);

print('Each iteration will keep f2 lower than some values between f2_min and f2_max, so ['       + str(f2_min) + ', ' + str(f2_max) + ']')

n = 4
step = int((f2_max - f2_min) / n)
steps = list(range(int(f2_min),int(f2_max),step)) + [f2_max]

x1_l = []
x2_l = []
for i in steps:
    model.e = i
    solver.solve(model);
    x1_l.append(value(model.X1))
    x2_l.append(value(model.X2))
plt.plot(x1_l,x2_l,'o-.');
plt.title('inefficient Pareto-front');
plt.grid(True);


# ## apply augmented $\epsilon$-Constraint

# max   f2 + delta*epsilon <br>
#  s.t. f2 - s = e

model.del_component(model.O_f1)
model.del_component(model.O_f2)
model.del_component(model.C_epsilon)

model.delta = Param(initialize=0.00001)

model.s = Var(within=NonNegativeReals)

model.O_f1 = Objective(expr = model.f1 + model.delta * model.s, sense=maximize)

model.C_e = Constraint(expr = model.f2 - model.s == model.e)

x1_l = []
x2_l = []
for i in range(160,190,6):
    model.e = i
    solver.solve(model);
    x1_l.append(value(model.X1))
    x2_l.append(value(model.X2))
plt.plot(x1_l,x2_l,'o-.');
plt.title('efficient Pareto-front');
plt.grid(True);